In [56]:
import matplotlib.pyplot as plt
import wisardpkg as wp
import numpy as np
import pandas as pd
import time
import cv2
import os
from unary_coding import inverted_unary

from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

def clf_eval(name, y_true, y_pred, classes=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']):
    clf_matrix = confusion_matrix(y_true, y_pred, labels=classes)
    print(clf_matrix)
    
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            #images.append(gist.extract(img))
            images.append(img)
    return images

def import_data(classes):
  first = True
  if "Expiro" in classes: #### VIRUS # EXPIRO
    img = load_images_from_folder('malevis_train_val_224x224/train/Expiro')
    if first: # FIRST?
      length = len(img)
      data = img
      y = ['Expiro']*length
      first = False
    else:
      data = np.concatenate((data, img), axis=0)
      y = np.concatenate((y,['Expiro']*length),axis=0)
    del img

  if "Neshta" in classes: #### VIRUS # NESHTA
    img = load_images_from_folder('malevis_train_val_224x224/train/Neshta')
    if first: # FIRST?
      length = len(img)
      data = img
      y = ['Neshta']*length
      first = False
    else:
      data = np.concatenate((data, img), axis=0)
      y = np.concatenate((y,['Neshta']*length),axis=0)
    del img

  if "Sality" in classes: #### VIRUS SALITY
    img = load_images_from_folder('malevis_train_val_224x224/train/Sality')
    if first: # FIRST?
      length = len(img)
      data = img
      y = ['Sality']*length
      first = False
    else:
      data = np.concatenate((data, img), axis=0)
      y = np.concatenate((y,['Sality']*length),axis=0)
    del img

  if "VBA" in classes: #### VIRUS VBA
    img = load_images_from_folder('malevis_train_val_224x224/train/VBA')
    if first: # FIRST?
      length = len(img)
      data = img
      y = ['VBA']*length
      first = False
    else:
      data = np.concatenate((data, img), axis=0)
      y = np.concatenate((y,['VBA']*length),axis=0)
    del img

  data = np.resize(data, (len(data),224*224*3))

  X = list(data)
  y = list(y)
  
  SPLIT_SIZE = 0.3
  X_traincv, X_testcv, y_traincv, y_testcv = model_selection.train_test_split(X,
                                                                              y,
                                                                              test_size=SPLIT_SIZE,
                                                                              random_state=0)

  return X_traincv, X_testcv, y_traincv, y_testcv


In [12]:
X_traincv, X_testcv, y_traincv, y_testcv = import_data(["Expiro","Neshta","Sality"])

In [54]:
np.max(X_traincv[0])

255

In [55]:
np.min(X_traincv[0])

0

In [73]:
wsd = wp.Wisard(20, bleachingActivated = True, ignoreZero = False, verbose = True)

In [ ]:
for i in range(len(X_traincv)):
    X = np.zeros(224*224*3*256)
    for j in range(len(X_traincv[i])):
        for k in range(1,X_traincv[i][j]):
            X[256*(j+1)-k] = 1
    wsd.train([list(X)],[y_traincv[i]])

In [25]:
pca = PCA(n_components=4)

In [26]:
X_train = X_traincv - np.mean(X_traincv)

In [27]:
X_new = pca.fit_transform(X_train)

In [28]:
print(pca.explained_variance_ratio_)

[0.16691316 0.04488565 0.03351217 0.02494754]


In [29]:
X_new.shape

(735, 4)

In [30]:
np.array(X_train).shape

(735, 150528)

In [31]:
wsd = wp.Wisard(20, bleachingActivated = True, ignoreZero = False, verbose = True)

In [37]:
X = (X_new - np.mean(X_new))/(np.max(X_new) - np.min(X_new))

In [38]:
X[0:10]

array([[-0.03199584, -0.06898544,  0.04696786,  0.02041766],
       [-0.10314314,  0.01531475,  0.01136893, -0.00449775],
       [-0.04347444,  0.03048707,  0.01186936, -0.0632046 ],
       [ 0.01494879, -0.0081407 , -0.02019321, -0.05055189],
       [-0.10191095,  0.00331683,  0.03069353, -0.01268776],
       [-0.00176801,  0.06763123, -0.0632608 ,  0.05625272],
       [ 0.16367182, -0.0624849 , -0.05857652, -0.00058742],
       [ 0.03999318, -0.23321941,  0.11594632,  0.11438134],
       [-0.11773556, -0.03591256, -0.0025728 , -0.04386888],
       [-0.01945295,  0.07296571,  0.02367647,  0.06000183]])

In [45]:
wsd = wp.Wisard(20, bleachingActivated = True, ignoreZero = False, verbose = True)

In [47]:
wsd.train(X, y_traincv)

TypeError: train(): incompatible function arguments. The following argument types are supported:
    1. (self: wisardpkg.Wisard, arg0: List[List[int]], arg1: List[str]) -> None
    2. (self: wisardpkg.Wisard, arg0: wisardpkg.DataSet) -> None

Invoked with: <wisardpkg.Wisard object at 0x7f0751fe5430>, array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), ['Neshta', 'Expiro', 'Sality', 'Neshta', 'Expiro', 'Sality', 'Neshta', 'Sality', 'Neshta', 'Sality', 'Sality', 'Expiro', 'Sality', 'Neshta', 'Expiro', 'Expiro', 'Neshta', 'Expiro', 'Expiro', 'Sality', 'Sality', 'Sality', 'Expiro', 'Neshta', 'Neshta', 'Sality', 'Expiro', 'Expiro', 'Expiro', 'Expiro', 'Sality', 'Sality', 'Sality', 'Expiro', 'Neshta', 'Neshta', 'Expiro', 'Sality', 'Sality', 'Sality', 'Sality', 'Expiro', 'Expiro', 'Neshta', 'Sality', 'Neshta', 'Neshta', 'Expiro', 'Neshta', 'Expiro', 'Expiro', 'Expiro', 'Neshta', 'Ne

In [ ]:

print("Training...")


print("Getting patterns...")

patterns = wsd.getMentalImages()

expiro = patterns["Expiro"]
neshta = patterns["Neshta"]
sality = patterns["Sality"]
vba = patterns["VBA"]

expiro = np.where(expiro > np.mean(expiro), 1, 0)
neshta = np.where(neshta > np.mean(neshta), 1, 0)
sality = np.where(sality > np.mean(sality), 1, 0)
vba = np.where(vba > np.mean(vba), 1, 0)

wsd2 = wp.Wisard(20, bleachingActivated = True, ignoreZero = False, verbose = True)

wsd2.train([expiro],["Expiro"])
wsd2.train([neshta],["Neshta"])
wsd2.train([sality],["Sality"])
wsd2.train([vba],["VBA"])

trained = 0
used = 0
for i in range(len(X_traincv)):
    out = wsd2.classify([X_traincv[i]])
    trained += 1
    print(out[0])
    print(y_traincv[i])
    if out[0] == y_traincv[i]:
        wsd2.train([X_traincv[i]], [y_traincv[i]])
        used += 1
    else:
        if random.random() < 0.3:
            wsd2.train([X_traincv[i]], [y_traincv[i]])
            used += 1

out = wsd2.classify(X_testcv)
print("Tried to train "+str(trained)+", but trained "+str(used)+"\n")
clf_eval('forget.png', y_testcv, out, classes = list(dict.fromkeys(y_testcv)))